#### @prompt example

In [1]:
from openbb import obb
import os
from magentic import prompt
from openbb import obb
obb.account.login(pat=os.environ['PAT_KEY'])
FMP_KEY = os.environ['FMP_KEY']


## Six Step Dance via Langchain
1. Taking into Account Company Industry (Growth / Value)
    1. Growth / Value
    2. Sector outperforming market or not
    3. Company outperforming sector
    4. PE vs Industry
2. Determining Company Financial Stability
    1. Debt To Equity
    2. Current Ratio
    3. Interest Margin Cover
3. Looking at Historical Earning and Growth
    1. Sales Growth
    2. EPS Growth
    3. Return on Invested Capital
    4. Profit Margin
    5. Cashflow Growth
    6. Equity Growth
    7. Return on Equity
4. Understanding Earnings and Sales Expectations
    1. Analyst Estimates
    2. PEG
5. Checking out competition
    1. P/E
    2. P/S
    3.MOAT
6. Estimating Company Values
    1. Insider Buying
    2. Senate TRading
    3. 13f

## Buffett Criteria
### Gross Margin Indicates profitability > 40% GrossProfit/Revenue GROSS PM close to 40 for recent years
### SG&A Margin efficiency in managing overhead costs < 30% SG&A Expenses / Gross Profit
### R&D Margin r&d spending relative to profitability < 30% R&D Expenses / Gross Profit
### Interest Expense Margin reliance on debt and financial health < 15 % Interest Expense / Operating Income
### Income Tax Margin Gauges tax efficiency ~ 20% Income Tax / Pre-Tax Income
### Profit Margin Shows overall profitabilty relative to revenue > 20% Net Income / Revenue
### EPS Growth Indicates growth and consistency in earnings >0 and growing Net Income / Shares Outstanding




In [2]:
obb.equity.fundamental.income(symbol='AAPL', limit=3, provider='yfinance').to_llm()

'[{"period_ending":"2024-09-30T00:00:00","operating_revenue":391035000000.0,"total_revenue":391035000000.0,"cost_of_revenue":210352000000.0,"gross_profit":180683000000.0,"selling_general_and_admin_expense":26097000000.0,"research_and_development_expense":31370000000.0,"operating_expense":57467000000.0,"operating_income":123216000000.0,"other_non_operating_income_expenses":269000000.0,"other_income_expense":269000000.0,"total_pre_tax_income":123485000000.0,"tax_provision":29749000000.0,"net_income_continuous_operations":93736000000.0,"net_income_including_noncontrolling_interests":93736000000.0,"net_income":93736000000.0,"net_income_attributable_to_common_shareholders":93736000000.0,"diluted_ni_availto_com_stockholders":93736000000.0,"basic_earnings_per_share":6.11,"diluted_earnings_per_share":6.08,"weighted_average_basic_shares_outstanding":15343783000.0,"weighted_average_diluted_shares_outstanding":15408095000.0,"total_operating_income_as_reported":123216000000.0,"total_expenses":2678

### Attempting to write a Small Agent

In [3]:
from langchain_openai import ChatOpenAI
from langchain.agents import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [4]:
@tool
def get_industry_performance(group:str) -> list:
    """ Return performance by industry for last week, last month, last quarter, last half year and last year"""
    return obb.equity.compare.groups(group='industry', metric='performance').to_llm()

@tool
def get_strong_buy_for_sector(sector : str) -> list :
    """ Return the strong buy recommendation for a given sector"""
    new_sector = '_'.join(sector.lower().split())
    data = obb.equity.screener(provider='finviz', sector=new_sector, recommendation='buy')
    return data.to_llm()

@tool
def get_best_stock_performers_for_sector(sector:str) -> list :
    """ Return the best  5 stock performers for last week and last month for a given sector"""
    data = obb.equity.screener(provider='finviz', filters_dict={'Sector' : sector, 'Performance' : 'Week Up', 'Performance 2' : 'Month Up'}, limit=5)
    return data.to_llm()

@tool
def get_best_stock_performers_for_industry(industry:str) -> list :
    """ Return the best  5 stock performers for last week and last month for an industry"""
    data = obb.equity.screener(provider='finviz', filters_dict={'Industry' : industry, 'Performance' : 'Week Up', 'Performance 2' : 'Month Up'}, limit=3)
    return data.to_llm()

@tool
def get_valuation_for_sectors(input:str) -> list:
    """ Return valuation metrics for the sector provided as input"""
    data = obb.equity.compare.groups(group='sector', metric='valuation', provider='finviz').to_df()
    
    filtered =  data[data.name == input]
    return filtered.to_json(
            orient="records",
            date_format="iso",
            date_unit="s",
        )

@tool
def get_valuation_for_industries(input:str) -> list:
    """ Return valuation meetrics for the industry provided as input"""
    data =  obb.equity.compare.groups(group='industry', metric='valuation', provider='finviz').to_df()
    filtered =  data[data.name == input]
    return filtered.to_json(
            orient="records",
            date_format="iso",
            date_unit="s",
        )


@tool
def get_valuation_for_company(ticker:str) -> list:
    """ Return Valuation metrics for a company"""
    # requires obb login
    obb.account.login(pat=os.environ['PAT_KEY'])
    return obb.equity.fundamental.ratios(symbol='AAPL', provider='fmp', limit=1).to_llm()

@tool
def get_income_statement(ticker:str) -> list:
    """ Return the last 3  income statement for  a given ticker"""
    return obb.equity.fundamental.income(symbol=ticker, limit=3, provider='yfinance').to_llm()



get_strong_buy_for_sector.invoke('Consumer Cyclical')
#get_valuation_for_sectors.invoke('Utilities')
#get_valuation_for_company.invoke('AAPL')
#get_valuation_for_industries.invoke('Medical Distribution')
#get_best_stock_performers_for_industry('Utilities')

'[{"symbol":"LI","name":"Li Auto Inc ADR","country":"China","sector":"Consumer Cyclical","industry":"Auto Manufacturers","market_cap":18340000000.0,"price":22.25,"change_percent":0.0525,"volume":10876244,"price_to_earnings":21.21},{"symbol":"RRGB","name":"Red Robin Gourmet Burgers Inc","country":"USA","sector":"Consumer Cyclical","industry":"Restaurants","market_cap":53450000.0,"price":3.06,"change_percent":0.0408,"volume":238978,"price_to_earnings":null},{"symbol":"UXIN","name":"Uxin Ltd ADR","country":"China","sector":"Consumer Cyclical","industry":"Auto & Truck Dealerships","market_cap":659210000.0,"price":3.51,"change_percent":0.0354,"volume":14217,"price_to_earnings":null},{"symbol":"XPEV","name":"XPeng Inc ADR","country":"China","sector":"Consumer Cyclical","industry":"Auto Manufacturers","market_cap":13950000000.0,"price":17.97,"change_percent":0.0304,"volume":19675600,"price_to_earnings":null},{"symbol":"ROL","name":"Rollins Inc","country":"USA","sector":"Consumer Cyclical","in

In [5]:
### instructions
instruction = """You are a helpful chatbot that can interact with an SQL database
for a computer store. You will take the users questions and turn them into SQL
queries using the tools available. Once you have the information you need, you will
answer the user's question using the data returned.

Use list_tables to see what tables are present, describe_table to understand thes
schema, and execute_query to issue an SQL SELECT query."""

In [6]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """ You are very powerful stock financial analys, but don't know current events,
                so you will use the tools available to answer any user question""",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [7]:
tools = [get_industry_performance, get_income_statement, get_best_stock_performers_for_industry, get_valuation_for_industries, get_income_statement]
llm_with_tools = llm.bind_tools(tools)

In [8]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

### Chat Memory

In [9]:
from langchain_core.prompts import MessagesPlaceholder
from langchain.memory import ConversationTokenBufferMemory
#from langchain_core.messages import count_tokens_approximately
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser
from langchain.agents import AgentExecutor
from langchain_core.messages import AIMessage, HumanMessage




MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """ You are very powerful stock financial analys, but don't know current events. 
                You  will take the user questions and answer using the tools available.
                Once you have the information you  need, you will answer user's questions using the data returned""",
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [10]:
tools = [get_industry_performance, get_best_stock_performers_for_industry, get_valuation_for_industries]
llm_with_tools = llm.bind_tools(tools)

In [11]:
chat_history = []
chat_history.append(HumanMessage(content="Your question here"))
chat_history.append(AIMessage(content="AI response here"))
memory = ConversationTokenBufferMemory(
    llm=llm,  # Required for token counting
    max_token_limit=14000,  # Leave buffer for functions + responses
    memory_key="chat_history",  # Must match your prompt's key
    return_messages=True
)

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: memory.load_memory_variables(x)["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

/tmp/ipykernel_48336/1189149751.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationTokenBufferMemory(


In [12]:
input1 = '''
First, Find the industry that has shown a constant postive performance across quarter, month and week.
Second, extract the valuation metrics for this industry.
Lastly, summarize your finding using not more than 200 words:

'''
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
print(result['output'])



> Entering new AgentExecutor chain...

Invoking: `get_industry_performance` with `{'group': 'all'}`


[{"name":"Broadcasting","performance_1d":-0.1012,"performance_1w":-0.2586,"performance_1m":-0.3293,"performance_3m":-0.3764,"performance_6m":-0.2845,"performance_1y":-0.3075,"performance_ytd":-0.2839,"analyst_recommendation":1.92,"volume":26630000,"volume_average":38160000,"volume_relative":0.7},{"name":"Oil & Gas Drilling","performance_1d":-0.1231,"performance_1w":-0.1634,"performance_1m":-0.2423,"performance_3m":-0.4147,"performance_6m":-0.4485,"performance_1y":-0.5724,"performance_ytd":-0.4002,"analyst_recommendation":1.89,"volume":78330000,"volume_average":61380000,"volume_relative":1.28},{"name":"Mortgage Finance","performance_1d":-0.0525,"performance_1w":-0.1473,"performance_1m":-0.052,"performance_3m":-0.0183,"performance_6m":-0.1304,"performance_1y":0.0049,"performance_ytd":-0.0503,"analyst_recommendation":2.47,"volume":28960000,"volume_average":16200000,"volume_relative":1.7

### Let's try to do another approach, We start with the industry, find the ones that are performing better than the industry and narrow down by criteria

In [22]:
input1 = '''
First, analyze the available data to find a industry that has consistently shown positive performance across quarterly, monthly, and weekly timeframes. 
Second, once you have identified the industry, extract its relevant valuation metrics (e.g., P/E, P/B, EV/EBITDA). 
Third, analyze the individual companies within the identified industry to determine the top performers based on relevant metrics like stock price appreciation and revenue growth. 
Fourth, extract the income statement for  the top performer. 
Finally, using income statement data, compute the revenue growth for the past 3 years.'''
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
print(result['output'])



> Entering new AgentExecutor chain...

Invoking: `get_valuation_for_sectors` with `{'input': 'all'}`


[]
Invoking: `get_valuation_for_industries` with `{'input': 'all'}`
responded: I couldn't retrieve the valuation metrics for all industries at once. Let me try to get the valuation metrics for each industry individually to identify the industry that has consistently shown positive performance across quarterly, monthly, and weekly timeframes.

[]
Invoking: `get_valuation_for_industries` with `{'input': 'quarterly'}`
responded: It seems I am unable to retrieve the valuation metrics for all industries at once. Let me try to get the valuation metrics for each industry individually to identify the industry that has consistently shown positive performance across quarterly, monthly, and weekly timeframes.

[]
Invoking: `get_valuation_for_industries` with `{'input': 'monthly'}`
responded: It seems I am unable to retrieve the valuation metrics for all industries at once. Let me try to get th

OpenBBError: 
[Unexpected Error] -> ValueError -> Invalid filter option 'technology'. Possible filter options: ['Any', 'Basic Materials', 'Communication Services', 'Consumer Cyclical', 'Consumer Defensive', 'Energy', 'Financial', 'Healthcare', 'Industrials', 'Real Estate', 'Technology', 'Utilities']

In [ ]:
input1 = "Using the recently found strong buys for Utilities , find their valuation metrics and compare it with the Utilities sector valuation metrics"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
print(result['output'])

In [ ]:
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
agent_executor.invoke({"input": "Which stock are recommented for this sector best performing sector", "chat_history": chat_history})